In [ ]:
import json
import glob
from os import listdir
from os.path import isfile, join
import os.path as osp
import copy
import os
import pickle
from datetime import datetime
import random
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.filtering.log.attributes import attributes_filter
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv
from torch_geometric.nn import GATv2Conv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
class SkinbaronDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super(SkinbaronDataset, self).__init__(root, transform, pre_transform, pre_filter)
    @property
    def processed_file_names(self):
        return processed_graphs
    def len(self):
        return len(self.processed_file_names)
    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [ ]:
# define architecture of GAE
class GATL1noSelf(nn.Module):
    """Define GATL1noSelf class for the Graph Attention neural network.(GAT)
    with one layer, and without self loops"""

    def __init__(self, size_in, size_out, edge_size, num_head, tmax, dropout_prob):
        super(GATL1noSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_out, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred

class GATL1withSelf(nn.Module):
    """Define GATL1withSelf class for the Graph Attention neural network.(GAT)
    with one layer, and self loops"""

    def __init__(self, size_in, size_out, edge_size, num_head, tmax, dropout_prob):
        super(GATL1withSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_out, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred
    

class GATL2noSelf(nn.Module):
    """Define GATL2noSelf class for the Graph Attention neural network.(GAT)
    with two layers, and without self loops"""

    def __init__(self, size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob):
        super(GATL2noSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_hid1, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.conv2 = GATv2Conv(in_channels=num_head * size_hid1, out_channels=size_out, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv2(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred
    
class GATL2withSelf(nn.Module):
    """Define GATL2withSelf class for the Graph Attention neural network.(GAT)
    with two layers, and self loops"""

    def __init__(self, size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob):
        super(GATL2withSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_hid1, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.conv2 = GATv2Conv(in_channels=num_head * size_hid1, out_channels=size_out, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv2(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred
    
class GATL3noSelf(nn.Module):
    """Define GATL3noSelf class for the Graph Attention neural network.(GAT)
    with three layers, and without self loops"""

    def __init__(self, size_in, size_out, size_hid1, size_hid2, edge_size, num_head, tmax, dropout_prob):
        super(GATL3noSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_hid1, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.conv2 = GATv2Conv(in_channels=num_head * size_hid1, out_channels=size_hid2, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.conv3 = GATv2Conv(in_channels=num_head * size_hid2, out_channels=size_out, heads=num_head, dropout=dropout_prob,
                               add_self_loops=False, edge_dim=edge_size)
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv2(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv3(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred
    
class GATL3withSelf(nn.Module):
    """Define GATL3withSelf class for the Graph Attention neural network.(GAT)
    with three layers, and without self loops"""

    def __init__(self, size_in, size_out, size_hid1, size_hid2, edge_size, num_head, tmax, dropout_prob):
        super(GATL3withSelf, self).__init__()
        self.conv1 = GATv2Conv(in_channels=size_in, out_channels=size_hid1, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.conv2 = GATv2Conv(in_channels=num_head * size_hid1, out_channels=size_hid2, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.conv3 = GATv2Conv(in_channels=num_head * size_hid2, out_channels=size_out, heads=num_head, edge_dim=edge_size,
                               dropout=dropout_prob, fill_value='mean')
        self.readout = nn.Linear(2 * num_head * size_out, tmax - 1)

    def forward(self, graph):
        graph.x = F.elu(self.conv1(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv2(graph.x, graph.edge_index, graph.edge_attr))
        graph.x = F.elu(self.conv3(graph.x, graph.edge_index, graph.edge_attr))
        s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, 0])], graph.x[int(graph.edge_index[1, 0])]))
        m_array_pred = torch.sigmoid(self.readout(s_t_nodes))
        for i in range(1, graph.edge_index.shape[1]):  # iterate over the number of edges in the batch (called graph)
            s_t_nodes = torch.cat((graph.x[int(graph.edge_index[0, i])], graph.x[int(graph.edge_index[1, i])]))
            edge_label_pred = torch.sigmoid(self.readout(s_t_nodes))
            m_array_pred = torch.cat((m_array_pred, edge_label_pred), 0)
        m_array_pred = m_array_pred.view(-1, tmax - 1)
        return m_array_pred

In [ ]:
# set the device to GPU or CPU (only if GPU is not available we will use CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# for reproducibility of the result (note: we use SGD for optimization)
np.random.seed(0)
# Define binary cross entropy loss.
batch_loss = nn.BCELoss()
#batch_loss = nn.MSELoss() # in case we use sum of time and control-flow in m-array
targat_path = r"D:\Final master thesis evaluation\exp1\data"
processed_Pattern = r"D:\Final master thesis evaluation\exp1\data\processed\*.pt"
# creating dataset
processed_graphs = glob.glob(processed_Pattern)
dataset = SkinbaronDataset(root=targat_path)
# train/validation/test split
train_dataset = dataset[0:9000]
validation_dataset = dataset[9000:12000]
test_dataset = dataset[12000:15000]
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(validation_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')
# Retrieving the case_id list from saved file on the disk:
case_id_file = open(case_id_target, "rb")
case_id_list = pickle.load(case_id_file)
case_id_file.close()
# find the length of the longest trace based on the edge_m_array attribute
tmax = int(dataset.get(0).edge_m_array.shape[1]) + 1
num_configurations = 54
active_arms = list(range(num_configurations))
hpo_path = r'D:\Final master thesis evaluation\exp2\HPO models'
hpo_models_target = []
for idx in range (num_configurations):
    hpo_models_target.append(osp.join(hpo_path, f'model_{idx}.pt'))
first_bracket_path = r'D:\Final master thesis evaluation\exp2\HPO models\first_bracket.pkl'
second_bracket_path = r'D:\Final master thesis evaluation\exp2\HPO models\second_bracket.pkl'
third_bracket_path = r'D:\Final master thesis evaluation\exp2\HPO models\third_bracket.pkl'
layer_list = []
self_loop_list = []
dropout_list = []
batch_size_list = []
bottleneck_list = []
learning_rate_list = []
random.seed(42)
for j in range (num_configurations):
    n1 = random.random()
    if n1 > 0.5:
        layer_list.append(2)
    else:
        layer_list.append(1)
    n2 = random.random()
    if n2 > 0.5:
        self_loop_list.append(1)
    else:
        self_loop_list.append(0)
    n3 = random.random()  
    if n3 <= 0.33:
        dropout_list.append(0.0)
    elif n3 <= 0.67:
        dropout_list.append(0.2)
    else:
        dropout_list.append(0.4)         
    n4 = random.random()
    if n4 <= 0.25:
        batch_size_list.append(16)
    elif n4 <= 0.5:
        batch_size_list.append(32)
    elif n4 <= 0.75:
        batch_size_list.append(64)
    else:
        batch_size_list.append(128)
    n5 = random.random()
    bottleneck_list.append(2*n5+1)
    n6 = random.random()
    learning_rate_list.append(np.power(10, (((n6-1)/0.5)-1)))
print(layer_list)
print(self_loop_list)
print(dropout_list)
print(batch_size_list)
print(bottleneck_list)
print(learning_rate_list)

In [ ]:
validation_id_list = case_id_list[9000:12000]
# import label dataframe
label_csv = pd.read_csv(r'D:\Final master thesis evaluation\exp1\large-0.1-1.csv')
label_csv.drop("Unnamed: 0", axis=1, inplace=True)
label_csv["case_id"] = pd.to_numeric(label_csv["case_id"])
label_csv.case_id.astype(str).astype(int)
label_csv.label.astype(str)
#label_csv.dtypes
validation_csv = label_csv.loc[label_csv['case_id'] > 9000]
validation_csv = validation_csv.loc[label_csv['case_id'] < 12001]
#print(len(validation_csv))
label_csv1 = validation_csv.loc[validation_csv['label']== 'normal']
normal_class = len(label_csv1)
anom_class = 3000 - len(label_csv1)
hpo_result = []
bracket = 0
for arm in active_arms:
    print(arm)
    # initialize the model
    num_head = 1
    size_in = dataset.get(0).x.shape[1]
    size_out = int(bottleneck_list[arm]* dataset.get(0).x.shape[1] / num_head)
    edge_size = dataset.get(0).edge_attr.shape[1]        
    size_hid1 = int((bottleneck_list[arm]+1)/2 * dataset.get(0).x.shape[1] / num_head)
    dropout_prob = dropout_list[arm]        
    if layer_list[arm] == 1 and self_loop_list[arm] == 0:
        model = GATL1noSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 1 and self_loop_list[arm] == 1:
        model = GATL1withSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 0:
        model = GATL2noSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 1:
        model = GATL2withSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    # setting the minibatch size
    batch_size = batch_size_list[arm]
    loader = DataLoader(train_dataset, batch_size=batch_size)
    # move to GPU (if available)
    model = model.to(device)
    # inizialize the optimizer
    lr = learning_rate_list[arm]
    weight_decay = 5e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # training of the model
    epochs = 5
    train_hist = {}
    train_hist['loss'] = []
    # Initialize training
    if bracket == 0:
        for layer in model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    model.train()
    # training loop
    for epoch in range(epochs):
        for idx, data_batch in enumerate(loader):
            data_batch = data_batch.to(device)
            optimizer.zero_grad()
            loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
            loss.backward()
            optimizer.step()
            train_hist['loss'].append(loss.item())
    # save and load learned parameters
    torch.save(model.state_dict(), hpo_models_target[arm])  
    batch_size = 1
    loss_list = []
    loader = DataLoader(validation_dataset, batch_size=batch_size)
    for data_batch in loader:
        loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
        if loss > 0:
            loss_list.append(loss.item())
        else:
            loss_list.append(0)
    GAT2_dictionary = {'case_id': validation_id_list, 'Loss': loss_list}
    loss_df = pd.DataFrame(GAT2_dictionary)
    loss_df["case_id"] = pd.to_numeric(loss_df["case_id"])
    result = pd.merge(loss_df, validation_csv, on=["case_id"])
    sorted_result = result.sort_values(by=['Loss'], ascending = False, ignore_index = True)
    sorted_list = sorted_result['label'].tolist()
    predictions = []
    for j in range (len(sorted_list)):
        if sorted_list[j] == 'normal':
            predictions.append(0)
        else:
            predictions.append(1)
    prediction_array = np.array(predictions)
    best_score = 0
    for j in range (len(sorted_list)):
        current_alarms = prediction_array[0:j+1]
        current_normals = prediction_array[j+1:]
        positives = j+1
        true_positives = np.sum(current_alarms)
        false_positives = positives - true_positives
        negatives = 3000 - positives
        false_negatives = np.sum(current_normals)
        true_negatives = negatives - false_negatives
        precision = true_positives/(true_positives+false_positives)
        recall = true_positives/(true_positives+false_negatives)
        f1_score = 2*precision*recall/(precision+recall)
        if f1_score > best_score:
            best_score = f1_score
    hpo_result.append((arm,best_score))

In [ ]:
print(hpo_result)
hpo_file = open(first_bracket_path, "wb")
pickle.dump(hpo_result, hpo_file)
hpo_file.close()

In [ ]:
active_arms = [0, 4, 7, 9, 11, 17, 19, 22, 26, 29, 31, 34, 35, 36, 37, 38, 46, 48]
hpo_result = []
bracket = 1
for arm in active_arms:
    print(arm)
    # initialize the model
    num_head = 1
    size_in = dataset.get(0).x.shape[1]
    size_out = int(bottleneck_list[arm]* dataset.get(0).x.shape[1] / num_head)
    edge_size = dataset.get(0).edge_attr.shape[1]        
    size_hid1 = int((bottleneck_list[arm]+1)/2 * dataset.get(0).x.shape[1] / num_head)
    dropout_prob = dropout_list[arm]        
    if layer_list[arm] == 1 and self_loop_list[arm] == 0:
        model = GATL1noSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 1 and self_loop_list[arm] == 1:
        model = GATL1withSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 0:
        model = GATL2noSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 1:
        model = GATL2withSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    # setting the minibatch size
    batch_size = batch_size_list[arm]
    loader = DataLoader(train_dataset, batch_size=batch_size)
    # move to GPU (if available)
    model = model.to(device)
    # inizialize the optimizer
    lr = learning_rate_list[arm]
    weight_decay = 5e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # training of the model
    epochs = 15
    train_hist = {}
    train_hist['loss'] = []
    # Initialize training
    if bracket == 0:
        for layer in model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    model.train()
    # training loop
    for epoch in range(epochs):
        for idx, data_batch in enumerate(loader):
            data_batch = data_batch.to(device)
            optimizer.zero_grad()
            loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
            loss.backward()
            optimizer.step()
            train_hist['loss'].append(loss.item())
    # save and load learned parameters
    torch.save(model.state_dict(), hpo_models_target[arm])  
    batch_size = 1
    loss_list = []
    loader = DataLoader(validation_dataset, batch_size=batch_size)
    for data_batch in loader:
        loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
        if loss > 0:
            loss_list.append(loss.item())
        else:
            loss_list.append(0)
    GAT2_dictionary = {'case_id': validation_id_list, 'Loss': loss_list}
    loss_df = pd.DataFrame(GAT2_dictionary)
    loss_df["case_id"] = pd.to_numeric(loss_df["case_id"])
    result = pd.merge(loss_df, validation_csv, on=["case_id"])
    sorted_result = result.sort_values(by=['Loss'], ascending = False, ignore_index = True)
    sorted_list = sorted_result['label'].tolist()
    predictions = []
    for j in range (len(sorted_list)):
        if sorted_list[j] == 'normal':
            predictions.append(0)
        else:
            predictions.append(1)
    prediction_array = np.array(predictions)
    best_score = 0
    for j in range (len(sorted_list)):
        current_alarms = prediction_array[0:j+1]
        current_normals = prediction_array[j+1:]
        positives = j+1
        true_positives = np.sum(current_alarms)
        false_positives = positives - true_positives
        negatives = 3000 - positives
        false_negatives = np.sum(current_normals)
        true_negatives = negatives - false_negatives
        precision = true_positives/(true_positives+false_positives)
        recall = true_positives/(true_positives+false_negatives)
        f1_score = 2*precision*recall/(precision+recall)
        if f1_score > best_score:
            best_score = f1_score
    hpo_result.append((arm,best_score))
hpo_file = open(second_bracket_path, "wb")
pickle.dump(hpo_result, hpo_file)
hpo_file.close()

In [ ]:
print(hpo_result)

In [ ]:
active_arms = [7, 26, 36, 38, 46, 48]
hpo_result = []
bracket = 2
for arm in active_arms:
    print(arm)
    # initialize the model
    num_head = 1
    size_in = dataset.get(0).x.shape[1]
    size_out = int(bottleneck_list[arm]* dataset.get(0).x.shape[1] / num_head)
    edge_size = dataset.get(0).edge_attr.shape[1]        
    size_hid1 = int((bottleneck_list[arm]+1)/2 * dataset.get(0).x.shape[1] / num_head)
    dropout_prob = dropout_list[arm]        
    if layer_list[arm] == 1 and self_loop_list[arm] == 0:
        model = GATL1noSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 1 and self_loop_list[arm] == 1:
        model = GATL1withSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 0:
        model = GATL2noSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    if layer_list[arm] == 2 and self_loop_list[arm] == 1:
        model = GATL2withSelf(size_in, size_out, size_hid1, edge_size, num_head, tmax, dropout_prob)
        if bracket != 0:
            model.load_state_dict(torch.load(hpo_models_target[arm]))
    # setting the minibatch size
    batch_size = batch_size_list[arm]
    loader = DataLoader(train_dataset, batch_size=batch_size)
    # move to GPU (if available)
    model = model.to(device)
    # inizialize the optimizer
    lr = learning_rate_list[arm]
    weight_decay = 5e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    # training of the model
    epochs = 45
    train_hist = {}
    train_hist['loss'] = []
    # Initialize training
    if bracket == 0:
        for layer in model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    model.train()
    # training loop
    for epoch in range(epochs):
        for idx, data_batch in enumerate(loader):
            data_batch = data_batch.to(device)
            optimizer.zero_grad()
            loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
            loss.backward()
            optimizer.step()
            train_hist['loss'].append(loss.item())
    # save and load learned parameters
    torch.save(model.state_dict(), hpo_models_target[arm])  
    batch_size = 1
    loss_list = []
    loader = DataLoader(validation_dataset, batch_size=batch_size)
    for data_batch in loader:
        loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
        if loss > 0:
            loss_list.append(loss.item())
        else:
            loss_list.append(0)
    GAT2_dictionary = {'case_id': validation_id_list, 'Loss': loss_list}
    loss_df = pd.DataFrame(GAT2_dictionary)
    loss_df["case_id"] = pd.to_numeric(loss_df["case_id"])
    result = pd.merge(loss_df, validation_csv, on=["case_id"])
    sorted_result = result.sort_values(by=['Loss'], ascending = False, ignore_index = True)
    sorted_list = sorted_result['label'].tolist()
    predictions = []
    for j in range (len(sorted_list)):
        if sorted_list[j] == 'normal':
            predictions.append(0)
        else:
            predictions.append(1)
    prediction_array = np.array(predictions)
    best_score = 0
    for j in range (len(sorted_list)):
        current_alarms = prediction_array[0:j+1]
        current_normals = prediction_array[j+1:]
        positives = j+1
        true_positives = np.sum(current_alarms)
        false_positives = positives - true_positives
        negatives = 3000 - positives
        false_negatives = np.sum(current_normals)
        true_negatives = negatives - false_negatives
        precision = true_positives/(true_positives+false_positives)
        recall = true_positives/(true_positives+false_negatives)
        f1_score = 2*precision*recall/(precision+recall)
        if f1_score > best_score:
            best_score = f1_score
    hpo_result.append((arm,best_score))
hpo_file = open(third_bracket_path, "wb")
pickle.dump(hpo_result, hpo_file)
hpo_file.close()

In [ ]:
print(hpo_result)

In [ ]:
# get anomaly scores produced on test dataset only for the best hyperparameter configuration
num_head = 1
# initialize the model
size_in = dataset.get(0).x.shape[1]
size_out = int(bottleneck_list[26]* dataset.get(0).x.shape[1] / num_head)
edge_size = dataset.get(0).edge_attr.shape[1]
model = GATL1withSelf(size_in, size_out, edge_size, num_head, tmax, dropout_prob)
model.load_state_dict(torch.load(hpo_models_target[26]))
# get the result in a dataframe
batch_size = 1
loader = DataLoader(test_dataset, batch_size=batch_size)
test_id_list = case_id_list[12000:15000]
loss_list = []
for data_batch in loader:
    loss = batch_loss(model.forward(data_batch).float(), data_batch.edge_m_array.float())
    if loss > 0:
        loss_list.append(loss.item())
    else:
        loss_list.append(0)  
GAT2_dictionary = {'case_id': test_id_list, 'Loss': loss_list}
loss_df = pd.DataFrame(GAT2_dictionary)
loss_df.to_pickle(r'D:\Final master thesis evaluation\exp2\bestgatmodl_test_loss.pt')
loss_df["case_id"] = pd.to_numeric(loss_df["case_id"])
result = pd.merge(loss_df, test_csv, on=["case_id"])
sorted_result = result.sort_values(by=['Loss'], ascending = False, ignore_index = True)
sorted_list = sorted_result['label'].tolist()
predictions = []
for j in range (len(sorted_list)):
    if sorted_list[j] == 'normal':
        predictions.append(0)
    else:
        predictions.append(1)
prediction_array = np.array(predictions)
best_score = 0
for j in range (len(sorted_list)):
    current_alarms = prediction_array[0:j+1]
    current_normals = prediction_array[j+1:]
    positives = j+1
    true_positives = np.sum(current_alarms)
    false_positives = positives - true_positives
    negatives = 3000 - positives
    false_negatives = np.sum(current_normals)
    true_negatives = negatives - false_negatives
    precision = true_positives/(true_positives+false_positives)
    recall = true_positives/(true_positives+false_negatives)
    f1_score = 2*precision*recall/(precision+recall)
    if f1_score > best_score:
        best_score = f1_score
        precision_on_normal = true_negatives/(true_negatives+false_negatives)
        recall_on_normal = true_negatives/(true_negatives+false_positives)
        f1_score_on_normal = 2*precision_on_normal*recall_on_normal/(precision_on_normal+recall_on_normal)
        macro_f1_score = (f1_score + f1_score_on_normal)/2
        best_cut = [j, precision, recall, f1_score, precision_on_normal, recall_on_normal,
                    f1_score_on_normal, macro_f1_score] 
        fp_for_best = false_positives
print(best_cut, fp_for_best)